# NSW Temperature missing-value fill

- Author: Mathew Traini
- Date: 21/03/22
- Purpose: Proof-of-concept notebook to fill as many missing temperature values as possible (in an expedient manner)
Uses a combination of left-joining using the demand dataset to remove unnecessary temperature observations, 
Meteostat lookups, and short range Pandas forward- and backfilling. Results in a date/time matched dataset (to the demand data)
with only 3 missing temperature values for the entire range 2010-2021.

Only deals with NSW data for the moment.

In [164]:
import pandas as pd
from datetime import datetime, timedelta
from meteostat import Hourly


DATA_PATH = "../../data/H06-2021/d/"

state = "VIC"
#states = ["QLD","SA","VIC"]
meteostat_ids = {"QLD":"94575", "VIC":"94575", "SA":"94672"}
timezones = {"QLD":"Australia/Brisbane", "SA":"Australia/Adelaide", "VIC":"Australia/Melbourne"}
locations = {"QLD":"Brisbane Archerfield Airport", "SA":"Adelaide (Kent Town)", "VIC":"Melbourne (Olympic Park)"}

# Set the MeteoStat ID for Bankstown
#bankstown_id = "94765"

# Timezone for Sydney
tz = "Australia/Sydney"

In [165]:
#for state in states:
temp_filename = "temprature_"+state.lower()+".csv"
demand_filename = "totaldemand_"+state.lower()+".csv"

demand_df = pd.read_csv(DATA_PATH+demand_filename)
temp_df = pd.read_csv(DATA_PATH+temp_filename)

temp_df.DATETIME = pd.to_datetime(temp_df.DATETIME)
demand_df.DATETIME = pd.to_datetime(demand_df.DATETIME)

result = pd.merge(demand_df, temp_df, left_on=['DATETIME'], right_on=['DATETIME'], how='left')    

if (state == "VIC"):
    # VIC is missing all temp values before 2013/06/01, so drop everything before then
    result = result[result['DATETIME'] >= "2013-06-01"]
    

In [166]:
temp_df.shape

(141681, 3)

In [167]:
demand_df.shape

(196513, 3)

In [168]:
result.shape

(136657, 5)

In [169]:
# Find any cases where there's missing temperature values
result[result['TEMPERATURE'].isnull()]

,DATETIME,TOTALDEMAND,REGIONID,LOCATION,TEMPERATURE
59856,2013-06-01 00:00:00,5781.73,VIC1,NaN,NaN
59857,2013-06-01 00:30:00,5430.38,VIC1,NaN,NaN
59858,2013-06-01 01:00:00,5149.84,VIC1,NaN,NaN
59859,2013-06-01 01:30:00,5105.74,VIC1,NaN,NaN
59860,2013-06-01 02:00:00,4939.19,VIC1,NaN,NaN
...,...,...,...,...,...
196267,2021-03-12 21:30:00,5106.15,VIC1,NaN,NaN
196268,2021-03-12 22:00:00,4911.38,VIC1,NaN,NaN
196269,2021-03-12 22:30:00,4751.75,VIC1,NaN,NaN
196270,2021-03-12 23:00:00,4689.39,VIC1,NaN,NaN


In [170]:
# Check for duplicate rows
result[result.duplicated()]

,DATETIME,TOTALDEMAND,REGIONID,LOCATION,TEMPERATURE


In [171]:
# Drop the duplicated rows
clean = result.drop_duplicates()
clean.shape

(136657, 5)

In [172]:
# Identify all the missing temperature values from the final clean version of the merged dataframe
clean[clean['TEMPERATURE'].isnull()]

,DATETIME,TOTALDEMAND,REGIONID,LOCATION,TEMPERATURE
59856,2013-06-01 00:00:00,5781.73,VIC1,NaN,NaN
59857,2013-06-01 00:30:00,5430.38,VIC1,NaN,NaN
59858,2013-06-01 01:00:00,5149.84,VIC1,NaN,NaN
59859,2013-06-01 01:30:00,5105.74,VIC1,NaN,NaN
59860,2013-06-01 02:00:00,4939.19,VIC1,NaN,NaN
...,...,...,...,...,...
196267,2021-03-12 21:30:00,5106.15,VIC1,NaN,NaN
196268,2021-03-12 22:00:00,4911.38,VIC1,NaN,NaN
196269,2021-03-12 22:30:00,4751.75,VIC1,NaN,NaN
196270,2021-03-12 23:00:00,4689.39,VIC1,NaN,NaN


In [173]:
# Get the row indices of all rows with missing temp values
missing_idx = clean.index[clean['TEMPERATURE'].isnull()]
len(missing_idx)

3045

In [176]:
# First iterate through the missing values (by row index) and attempt to find replacements in the MeteoStat database
# Not all timepoints will be present, but try to fill as many gaps as possible.
# MeteoStat only has readings on the hour...there are no :30 readings. Skip anything that is on the half-hour.

found = 0
missed = 0
import datetime 

for idx in missing_idx:
    #print(pd.clean.iloc[idx]['DATETIME'])
    # Set the start and end dates for the Meteostat query
    #print(idx)
    start_date = clean.loc[idx]['DATETIME'].to_pydatetime()
    #if (start_date.date() != datetime.date(2017,4,2)):
        

        # Only send the query if the missing value is on the hour
    if (start_date.minute == 0):
        end_date = start_date + timedelta(hours = 1)
        
            #print("On the hour, getting ",start_date, end_date)
    
        

            
        meteo_data = Hourly(loc=meteostat_ids[state], start=start_date, end=end_date, timezone=timezones[state])
        meteo_data = meteo_data.fetch()
        if (meteo_data.empty is False):
            clean.at[idx,'TEMPERATURE'] = meteo_data.iloc[0]['temp']
            found = found+1
        else:
            missed = missed + 1
            
print("Found/missed/total: ", found, missed, found+missed )
        


Found/missed/total:  1231 293 1524


In [179]:
# We now filled in as many missing values as MeteoStat can provide.
# Fill in as many remaining empty values using the built-in Pandas fillna method with a limit of 3 (ie: backfilling a maximum of 3 missed row)
backfilled = clean.fillna(method='bfill', limit=3)

# Now forward fill any missing values (up to a limit of 3)
backfilled = backfilled.fillna(method='ffill', limit=3)

# Finally set all LOCATION to 'Bankstown' (this is invariant for the entire NSW dataset)
backfilled['LOCATION'] = locations[state]

In [180]:
backfilled[backfilled['TEMPERATURE'].isnull()]

,DATETIME,TOTALDEMAND,REGIONID,LOCATION,TEMPERATURE
68796,2013-12-04 06:00:00,5232.26,VIC1,Melbourne (Olympic Park),NaN
68797,2013-12-04 06:30:00,5805.43,VIC1,Melbourne (Olympic Park),NaN
68798,2013-12-04 07:00:00,6076.35,VIC1,Melbourne (Olympic Park),NaN


In [181]:
# Write out the cleaned up temperature data in the same format as was originally supplied. This can then be used in pre-existing processing notebooks  
final = backfilled[['LOCATION','DATETIME','TEMPERATURE']]
final.to_csv('temperature_vic_cleaned.csv', index=False)

In [182]:
final.shape

(136657, 3)

In [183]:
final.isna().sum()

LOCATION       0
DATETIME       0
TEMPERATURE    3
dtype: int64